In [1]:
import pandas as pd

import pathlib as pl

import os

In [2]:
import shutil

In [3]:
data_folder = pl.Path("data")

labels_df = pd.read_csv(data_folder / "train.csv")

In [4]:
labels_df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [5]:
len(labels_df)

7095

In [6]:
labels_df.ClassId.value_counts()

3    5150
1     897
4     801
2     247
Name: ClassId, dtype: int64

In [7]:
labels_df.ImageId.value_counts()

db4867ee8.jpg    3
ef24da2ba.jpg    3
bdb31300b.jpg    2
ef8ba30c0.jpg    2
ea9f9985a.jpg    2
                ..
dc94faafc.jpg    1
268446e17.jpg    1
eaaa61b15.jpg    1
07b1a9644.jpg    1
b10ca1dc4.jpg    1
Name: ImageId, Length: 6666, dtype: int64

In [8]:
train_image_folder = data_folder / "train_images"

In [9]:
labeled_image_files = [
    f 
    for f in os.listdir(train_image_folder)
    if f.endswith(".jpg")
]

In [10]:
len(labeled_image_files)

12568

In [11]:
severstal_binary_folder = data_folder / "severstal-binary"

In [12]:
if severstal_binary_folder.exists():
    shutil.rmtree(severstal_binary_folder)

In [13]:
os.makedirs(severstal_binary_folder / "train" / "good")
os.makedirs(severstal_binary_folder / "train" / "bad")

os.makedirs(severstal_binary_folder / "validate" / "good")
os.makedirs(severstal_binary_folder / "validate" / "bad")

In [14]:
take_every_nth = 20  # to reduce disk space
labeled_image_files = [
    image 
    for i, image in enumerate(labeled_image_files)
    if i % take_every_nth == 0
]

In [15]:
train_validation_split = 0.2

In [16]:
n_training = int(len(labeled_image_files) * (1 - train_validation_split))

In [17]:
train_image_files = labeled_image_files[:n_training]
validation_image_files = labeled_image_files[n_training:]

In [25]:
for ti in train_image_files:
    assert not ti in validation_image_files

In [18]:
bad_images = labels_df.ImageId.values

for image in train_image_files:
    if image in bad_images:
        shutil.copy(train_image_folder / image, severstal_binary_folder / "train" / "bad" /image)
    else:
        shutil.copy(train_image_folder / image, severstal_binary_folder / "train" / "good" /image)

In [19]:
print("N bad training:", len(os.listdir(severstal_binary_folder / "train" / "bad")))
print("N good training:", len(os.listdir(severstal_binary_folder / "train" / "good")))

N bad training: 244
N good training: 259


In [21]:
for image in validation_image_files:
    if image in bad_images:
        shutil.copy(train_image_folder / image, severstal_binary_folder / "validate" / "bad" /image)
    else:
        shutil.copy(train_image_folder / image, severstal_binary_folder / "validate" / "good" /image)

In [22]:
print("N bad validation:", len(os.listdir(severstal_binary_folder / "validate" / "bad")))
print("N good validation:", len(os.listdir(severstal_binary_folder / "validate" / "good")))

N bad validation: 69
N good validation: 57
